In [2]:
!pip install osmnx

  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 112.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [osmnx]32m5/6 [osmnx]das]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install networkx


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [folium]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [geopy]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install requests


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install "qiskit[visualization]">=1.0 --user

In [7]:
!pip install qiskit-aer --user


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [11]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install qiskit


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [9]:
!pip install qiskit-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.5/646.5 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for docplex: filename=docplex-2.30.251-py3-none-any.whl size=685954 sha256=641633cf717a580901e75ae29fcdc76a56c36f5c8494e220398903bfc8f68ab3
  Stored in directory: /home/jovyan/.cache/pip/wheels/74/6a/00/0c08d9a7ffaa10ffd4462512d15d05197ac44d88436910bc4c
Successfully built docplex
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [qiskit-optimization]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [10]:
!pip install qiskit-algorithms


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
import folium
import itertools
from geopy.distance import geodesic
import time
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import requests
import json

# Configure settings
ox.settings.use_cache = True
ox.settings.log_console = True
import json
import math
import numpy as np
from itertools import permutations
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.converters import QuadraticProgramToQubo
from IPython.display import display, Math

In [13]:
# --- Cell 6b: Point-to-Point Best Route Helper Functions ---
def best_route(lat1, lon1, lat2, lon2, return_geometry=True):
    """
    Calculate the best driving route and distance between two points
    using OSRM (lat/lon coordinates).

    Parameters:
        lat1, lon1 : float - coordinates of point A
        lat2, lon2 : float - coordinates of point B
        return_geometry : bool - if True, also return route geometry

    Returns:
        distance_km : float
        duration_min : float
        geometry : list (only if return_geometry=True)
    """
    coords_str = f"{lon1},{lat1};{lon2},{lat2}"
    url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}?overview=full&geometries=geojson"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['code'] == 'Ok' and data['routes']:
            route = data['routes'][0]
            distance_km = route['distance'] / 1000  # meters → km
            duration_min = route['duration'] / 60   # seconds → minutes
            geometry = route['geometry']['coordinates']

            # Debug print
            print(f"Best route from ({lat1}, {lon1}) → ({lat2}, {lon2})")
            print(f"  Distance: {distance_km:.2f} km")
            print(f"  Duration: {duration_min:.2f} min")

            if return_geometry:
                return distance_km #, duration_min, geometry
            return distance_km, duration_min
        else:
            print("OSRM error: No route found")
            return None

    except requests.exceptions.RequestException as e:
        print(f"OSRM API error: {e}")
        return None

In [14]:
# --- Cell 2: Raw data (from JSON) ---
data = {
    "hospital": {
        "name": "Central Hospital",
        "latitude": 29.99512653425452,
        "longitude": 31.68462840171934,
        "type": "destination"
    },
    "patients": [
        {"id": "DT", "name": "Patient DT", "latitude": 30.000417586266437, "longitude": 31.73960813272627},
        {"id": "GR", "name": "Patient GR", "latitude": 30.011344405285193, "longitude": 31.747827362371993},
        {"id": "R2", "name": "Patient R2", "latitude": 30.030388325206854, "longitude": 31.669231198639675},
        {"id": "R3_2", "name": "Patient R3_2", "latitude": 30.030940768851426, "longitude": 31.688371339937028},
        {"id": "IT", "name": "Patient IT", "latitude": 30.01285635906825, "longitude": 31.693811715848444}
    ]
}

In [16]:
# Hospital is just a single dict
hospital = {
    "latitude": data["hospital"]["latitude"],
    "longitude": data["hospital"]["longitude"]
}

# Patients is already a list of dicts
patients = data["patients"]

n_patients = len(patients)
max_stops = 3  # Maximum stops per trip

# Create a list of all locations (hospital first, then patients)
locations = [hospital] + [{"latitude": p["latitude"], "longitude": p["longitude"]} for p in patients]
n_locations = len(locations)


In [17]:
# Precompute distance matrix
dist_matrix = np.zeros((n_locations, n_locations))
for i in range(n_locations):
    for j in range(n_locations):
        if i != j:
            coord_i = locations[i]
            coord_j = locations[j]
            dist_matrix[i, j] = best_route(
                coord_i["latitude"], coord_i["longitude"],
                coord_j["latitude"], coord_j["longitude"]
            )
latex_str = r"\begin{bmatrix}" + \
            r"\\".join([" & ".join(map(str, row)) for row in dist_matrix]) + \
            r"\end{bmatrix}"

display(Math(latex_str))

Best route from (29.99512653425452, 31.68462840171934) → (30.000417586266437, 31.73960813272627)
  Distance: 8.63 km
  Duration: 13.18 min
Best route from (29.99512653425452, 31.68462840171934) → (30.011344405285193, 31.747827362371993)
  Distance: 11.50 km
  Duration: 15.42 min
Best route from (29.99512653425452, 31.68462840171934) → (30.030388325206854, 31.669231198639675)
  Distance: 9.45 km
  Duration: 15.81 min
Best route from (29.99512653425452, 31.68462840171934) → (30.030940768851426, 31.688371339937028)
  Distance: 10.85 km
  Duration: 16.44 min
Best route from (29.99512653425452, 31.68462840171934) → (30.01285635906825, 31.693811715848444)
  Distance: 9.67 km
  Duration: 12.29 min
Best route from (30.000417586266437, 31.73960813272627) → (29.99512653425452, 31.68462840171934)
  Distance: 14.19 km
  Duration: 19.01 min
Best route from (30.000417586266437, 31.73960813272627) → (30.011344405285193, 31.747827362371993)
  Distance: 2.36 km
  Duration: 7.40 min
Best route from (30.

<IPython.core.display.Math object>

In [18]:

# Create Quadratic Program with a more efficient formulation
qp = QuadraticProgram(name="Ambulance_Routing")

In [19]:

# Add binary variables: x_{i,t} - patient i is in trip t
for i in range(n_patients):
    for t in range(2):  # We need at most 2 trips for 5 patients
        qp.binary_var(name=f"x_{i}_{t}")

# Add constraint: Each patient visited exactly once
for i in range(n_patients):
    constraint_terms = {}
    for t in range(2):
        constraint_terms[f"x_{i}_{t}"] = 1
    qp.linear_constraint(
        linear=constraint_terms,
        sense="==",
        rhs=1,
        name=f"patient_{i}_visited_once"
    )

# Add constraint: Maximum patients per trip
for t in range(2):
    constraint_terms = {}
    for i in range(n_patients):
        constraint_terms[f"x_{i}_{t}"] = 1
    qp.linear_constraint(
        linear=constraint_terms,
        sense="<=",
        rhs=max_stops,
        name=f"trip_{t}_max_patients"
    )

# Build the objective function
linear_terms = {}
quadratic_terms = {}

# Distance terms for each trip
for t in range(2):
    # Hospital to first patient distance (approximation)
    for i in range(n_patients):
        linear_terms[f"x_{i}_{t}"] = dist_matrix[0, i+1] * 0.5  # Weighted

    # Distances between patients in the same trip
    for i in range(n_patients):
        for j in range(i+1, n_patients):
            quadratic_terms[(f"x_{i}_{t}", f"x_{j}_{t}")] = dist_matrix[i+1, j+1] * 0.3  # Weighted

In [20]:
# Convert to QUBO for better performance
converter = QuadraticProgramToQubo()
qubo = converter.convert(qp)

#print("Problem formulation complete")

#print(f"Variables: {qubo.get_num_vars()}")
#print(f"Constraints: {qubo.get_num_linear_constraints()}")


In [21]:

# Solve with QAOA using a simplified approach
optimizer = COBYLA(maxiter=50)
qaoa = QAOA(sampler=StatevectorSampler(), optimizer=optimizer, reps=10)
algorithm = MinimumEigenOptimizer(qaoa)

result = algorithm.solve(qubo)
print("\nOptimization result:")


Optimization result:


In [22]:


# Interpret the solution
print("\nSuggested trips:")
trips = [[], []]  # Two trips
for i in range(n_patients):
    for t in range(2):
        var_name = f"x_{i}_{t}"
        if var_name in result.variables_dict and result.variables_dict[var_name] > 0.5:
            trips[t].append(patients[i]["id"])



Suggested trips:


In [23]:
# Calculate actual distances for each trip
def calculate_trip_distance(patient_ids):
    """Calculate the minimum distance for a trip visiting the given patients."""
    if not patient_ids:
        return 0, []

    # Get indices of patients in the distance matrix
    patient_indices = []
    for pid in patient_ids:
        for i, p in enumerate(patients):
            if p["id"] == pid:
                patient_indices.append(i+1)  # +1 because hospital is at index 0

    # Find the optimal order for this trip
    min_distance = float('inf')
    best_order = []

    # Try all permutations of patients to find the shortest path
    for order in permutations(patient_indices):
        distance = dist_matrix[0, order[0]]  # Hospital to first patient

        # Distances between patients
        for j in range(len(order)-1):
            distance += dist_matrix[order[j], order[j+1]]

        # Last patient back to hospital
        distance += dist_matrix[order[-1], 0]

        if distance < min_distance:
            min_distance = distance
            best_order = [patients[idx-1]["id"] for idx in order]

    return min_distance, best_order


In [25]:
total_distance = 0
for t in range(2):
    if trips[t]:
        distance, order = calculate_trip_distance(trips[t])
        total_distance += distance
        print(f"Trip {t+1}: Hospital -> {' -> '.join(order)} -> Hospital (Distance: {distance:.2f} km)")
    else:
        print(f"Trip {t+1}: Empty")

print(f"\nTotal distance: {total_distance:.2f} km")

print("\nPatient coordinates:")
for i, patient in enumerate(patients):
    print(f"{patient['id']}: ({patient['latitude']:.6f}, {patient['longitude']:.6f})")

print(f"Hospital: ({hospital['latitude']:.6f}, {hospital['longitude']:.6f})")


Trip 1: Hospital -> DT -> GR -> R2 -> Hospital (Distance: 34.66 km)
Trip 2: Hospital -> IT -> R3_2 -> Hospital (Distance: 25.74 km)

Total distance: 60.40 km

Patient coordinates:
DT: (30.000418, 31.739608)
GR: (30.011344, 31.747827)
R2: (30.030388, 31.669231)
R3_2: (30.030941, 31.688371)
IT: (30.012856, 31.693812)
Hospital: (29.995127, 31.684628)
